In [1]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns


c:\Users\Matin\Documents\GitHub\Examensarbete2025\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

In [4]:
df = pd.read_csv('../datasets/FA-KES-Dataset_cleaned.csv')


In [5]:
model_path = '../models/fake-news-roberta'
roberta = RobertaForSequenceClassification.from_pretrained(model_path, num_labels=2)
tokenizer = RobertaTokenizer.from_pretrained(model_path)


In [6]:
text = df['text'].values
text = [str(text) for text in text]
# 1500 length
text = [text[:1500] if len(text) > 1500 else text for text in text]


In [8]:
true_labels = df['label'].to_list()


In [10]:
#predict test dataset in batches
batch_size = 2
predicted_labels = []

for i in range(0, len(text), batch_size):
    batch_texts = text[i:i + batch_size]
    inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt')
    outputs = roberta(**inputs)

    logits = outputs.logits  # [batch_size, num_labels]
    batch_preds = torch.argmax(logits, dim=1)  # [batch_size]
    predicted_labels.extend(batch_preds.tolist())  # [0, 1, 2, ...]


In [11]:
y_pred = predicted_labels
y_true = true_labels

print(classification_report(y_true, y_pred))
#visualize the confusion matrix



              precision    recall  f1-score   support

           0       0.45      0.23      0.31       378
           1       0.52      0.75      0.62       426

    accuracy                           0.50       804
   macro avg       0.49      0.49      0.46       804
weighted avg       0.49      0.50      0.47       804

[[ 88 290]
 [108 318]]
